## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Busselton,AU,-33.65,115.33,54.00,96,22,7.92,few clouds
1,1,Vaini,TO,-21.20,-175.20,75.20,78,40,12.75,scattered clouds
2,2,Bontang,ID,0.13,117.50,76.84,86,96,6.71,overcast clouds
3,3,Thompson,CA,55.74,-97.86,26.60,63,75,9.17,broken clouds
4,4,Porto Walter,BR,-8.27,-72.74,92.80,47,97,2.33,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Vaini,TO,-21.20,-175.20,75.20,78,40,12.75,scattered clouds
2,2,Bontang,ID,0.13,117.50,76.84,86,96,6.71,overcast clouds
11,11,Hithadhoo,MV,-0.60,73.08,83.43,67,93,9.44,overcast clouds
15,15,Rikitea,PF,-23.12,-134.97,75.33,84,100,17.27,overcast clouds
17,17,Puerto Narino,CO,-3.77,-70.38,84.20,89,75,4.70,heavy intensity rain
22,22,Kloulklubed,PW,7.04,134.26,75.20,94,75,2.91,broken clouds
28,28,San Jose,US,37.34,-121.89,77.00,13,1,14.99,clear sky
31,31,Ewa Beach,US,21.32,-158.01,80.60,74,1,1.97,clear sky
32,32,Avera,US,33.19,-82.53,80.60,61,40,3.36,scattered clouds
34,34,Atuona,PF,-9.80,-139.03,79.05,72,2,20.56,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        199
City           199
Country        199
Lat            199
Lng            199
Max Temp       199
Humidity       199
Cloudiness     199
Wind Speed     199
Description    199
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,
2,Bontang,ID,76.84,overcast clouds,0.13,117.50,
11,Hithadhoo,MV,83.43,overcast clouds,-0.60,73.08,
15,Rikitea,PF,75.33,overcast clouds,-23.12,-134.97,
17,Puerto Narino,CO,84.20,heavy intensity rain,-3.77,-70.38,
22,Kloulklubed,PW,75.20,broken clouds,7.04,134.26,
28,San Jose,US,77.00,clear sky,37.34,-121.89,
31,Ewa Beach,US,80.60,clear sky,21.32,-158.01,
32,Avera,US,80.60,scattered clouds,33.19,-82.53,
34,Atuona,PF,79.05,clear sky,-9.80,-139.03,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df.isna()


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
11,False,False,False,False,False,False,False
15,False,False,False,False,False,False,False
17,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
654,False,False,False,False,False,False,False
655,False,False,False,False,False,False,False
658,False,False,False,False,False,False,False
662,False,False,False,False,False,False,False


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.count()

City           199
Country        199
Max Temp       199
Description    199
Lat            199
Lng            199
Hotel Name     199
dtype: int64

In [25]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))